# Prediction model

Importing Libraries

In [2]:
!pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Required imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import io
from IPython.display import display
import ipywidgets as widgets
import cv2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.utils import to_categorical
print("Loading and preprocessing data...")


Loading and preprocessing data...


## Training Data

In [ ]:
# Load the CSV files
train_data = pd.read_csv('emnist-digits-train.csv')
test_data = pd.read_csv('emnist-digits-test.csv')

# Separate features and labels
X_train = train_data.iloc[:, 1:].values
y_train = train_data.iloc[:, 0].values
X_test = test_data.iloc[:, 1:].values
y_test = test_data.iloc[:, 0].values

# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Reshape for CNN
X_train_cnn = X_train.reshape(-1, 28, 28, 1)
X_test_cnn = X_test.reshape(-1, 28, 28, 1)

# One-hot encoding for CNN
y_train_cnn = to_categorical(y_train)
y_test_cnn = to_categorical(y_test)

print("Training models...")

# Train KNN
print("Training KNN...")
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train.reshape(len(X_train), -1), y_train)

# Train SVM
print("Training SVM...")
svm = SVC(C=1.0, kernel='linear')
svm.fit(X_train.reshape(len(X_train), -1), y_train)

# Train Logistic Regression
print("Training Logistic Regression...")
log_reg = LogisticRegression(max_iter=500)
log_reg.fit(X_train.reshape(len(X_train), -1), y_train)

# Train Decision Tree
print("Training Decision Tree...")
decision_tree = DecisionTreeClassifier(max_depth=10)
decision_tree.fit(X_train.reshape(len(X_train), -1), y_train)

# Train CNN
print("Training CNN...")
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_cnn, y_train_cnn, epochs=3, batch_size=128, validation_data=(X_test_cnn, y_test_cnn), verbose=1)

# Store all models in a dictionary
models = {
    'knn': knn,
    'svm': svm,
    'log_reg': log_reg,
    'decision_tree': decision_tree,
    'cnn_model': cnn_model
}

print("All models trained successfully!")

Training models...
Training KNN...
Training SVM...


## preprocessing image

In [ ]:
def preprocess_image(image):
    """
    Preprocess the uploaded image to match the EMNIST format
    """
    # Convert to grayscale if needed
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Resize to 28x28
    image = cv2.resize(image, (28, 28))
    
    # Invert image (EMNIST digits are white on black)
    image = 255 - image
    
    # Normalize pixel values
    image = image / 255.0
    
    return image

## predicting model

In [ ]:
def predict_digit(image, models):
    """
    Make predictions using all trained models
    """
    # Prepare image for different model formats
    img_flat = image.reshape(1, -1)
    img_cnn = image.reshape(1, 28, 28, 1)
    
    predictions = {}
    
    # Get predictions from each model
    predictions['KNN'] = models['knn'].predict(img_flat)[0]
    predictions['SVM'] = models['svm'].predict(img_flat)[0]
    predictions['Logistic Regression'] = models['log_reg'].predict(img_flat)[0]
    predictions['Decision Tree'] = models['decision_tree'].predict(img_flat)[0]
    predictions['CNN'] = np.argmax(models['cnn_model'].predict(img_cnn), axis=1)[0]
    
    return predictions

## Plotting result

In [ ]:
def plot_prediction_results(image, predictions):
    """
    Display the image and predictions from all models
    """
    plt.figure(figsize=(12, 4))
    
    # Plot original image
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.title('Input Image')
    plt.axis('off')
    
    # Plot predictions
    plt.subplot(1, 2, 2)
    models = list(predictions.keys())
    preds = list(predictions.values())
    
    # Create a table with predictions
    cell_text = [[model, str(pred)] for model, pred in predictions.items()]
    plt.table(cellText=cell_text, 
             colLabels=['Model', 'Prediction'],
             loc='center',
             cellLoc='center')
    plt.axis('off')
    plt.title('Model Predictions')
    
    plt.tight_layout()
    plt.show()

## uploading files

In [ ]:
def handle_upload(change):
    """
    Handle the file upload and make predictions
    """
    try:
        # Get the uploaded file's content
        content = change.new[0].content
        
        # Convert to numpy array
        image = plt.imread(io.BytesIO(content))
        
        # Preprocess image
        processed_image = preprocess_image(image)
        
        # Get predictions
        predictions = predict_digit(processed_image, models)
        
        # Display results
        plot_prediction_results(processed_image, predictions)
        
    except Exception as e:
        print(f"Error processing image: {str(e)}")

## displaying result

In [ ]:
# Create file upload widget
upload_widget = widgets.FileUpload(
    accept='image/*',  # Accept only image files
    multiple=False,    # Only single file upload
    description='Upload Image'
)

# Register the callback
upload_widget.observe(handle_upload, names='value')

# Display the widget
display(upload_widget)

print("Upload an image of a handwritten digit to get predictions from all models!")